In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import collections  

In [5]:
url = f"https://api.opendota.com/api/teams"
response = requests.get(url)
teams_data = response.json()

In [6]:
tl_id = []
for teams in teams_data: 
    if teams['name'] == 'Team Liquid': 
        tl_id = teams['team_id']
        print(tl_id)

2163


In [7]:
def team_info(tl_id):
    url = f"https://api.opendota.com/api/teams/{tl_id}"
    response = requests.get(url)
    return response.json()

In [8]:
def win_rate(tl_id):
    team_data = team_info(tl_id)
    wins = team_data['wins']
    losses = team_data['losses']
    total = wins + losses
    win_rate = (wins / total) * 100 if total > 0 else 0
    return round(win_rate, 1)
print(f"A Falcons tem {win_rate(tl_id)}% de WinRate")

A Falcons tem 59.7% de WinRate


In [9]:
def get_matches(tl_id):
    url = f"https://api.opendota.com/api/teams/{tl_id}/matches"
    response = requests.get(url)
    matches = response.json()
    return matches[:4]

In [10]:
matches = get_matches(tl_id)

In [11]:
def format_duration(duration):
    hours = duration // 3600             
    minutes = (duration % 3600) // 60    
    seconds = duration % 60    
    if hours > 0:
        return f"{hours}h {minutes}m {seconds}s"
    else:
        return f"{minutes}m {seconds}s"

In [12]:
def team_side(match):
    if match['radiant']:
        return True  
    else:
        return False

In [13]:
for match in matches:
    is_radiant = team_side(match)
    if (is_radiant and match['radiant_win']) or (not is_radiant and not match['radiant_win']):
        resultado = 'Vitória'
    else:
        resultado = 'Derrota'
    duration_formatted = format_duration(match['duration'])
    print(f"Resultado: {resultado}, Duração: {duration_formatted}")

Resultado: Vitória, Duração: 32m 50s
Resultado: Derrota, Duração: 38m 34s
Resultado: Vitória, Duração: 35m 21s
Resultado: Derrota, Duração: 46m 22s


In [14]:
match_ids = []
for match in matches:
    match_ids.append(match['match_id']) 

In [15]:
count = 0
count1 = 1
total_kils = 0

for match in matches:
    is_radiant = team_side(match)
    radiant_score = match['radiant_score']
    dire_score = match['dire_score']
    total_score_match = radiant_score + dire_score
    total_kils += total_score_match 
    print(f"A partida {count1} teve {total_score_match} abates")
    if is_radiant == True:
        print(f"A Tundra teve {radiant_score} kills")
        print(f"O outro time teve {dire_score} kills")
    
    elif is_radiant == False:
        print(f"A Tundra teve {dire_score} kills")
        print(f"O outro time teve {radiant_score} kills")

    count += 1
    count1 += 1
    
media = (total_kils/count)
print(f"Media de kils nas ultimas {count} partidas foi de {media} kils")
print(f"Total de kills em todas as partidas: {total_kils}")

A partida 1 teve 50 abates
A Tundra teve 30 kills
O outro time teve 20 kills
A partida 2 teve 39 abates
A Tundra teve 11 kills
O outro time teve 28 kills
A partida 3 teve 46 abates
A Tundra teve 31 kills
O outro time teve 15 kills
A partida 4 teve 59 abates
A Tundra teve 26 kills
O outro time teve 33 kills
Media de kils nas ultimas 4 partidas foi de 48.5 kils
Total de kills em todas as partidas: 194


In [16]:
total_duration = 0
for match in matches:
    total_duration += match['duration']  
media = total_duration / 4
print(f"O tempo total de duracao das partidas foi de {format_duration(total_duration)}")  
print(f"A media de tempo de cada partida foi de {format_duration(media)}")  

O tempo total de duracao das partidas foi de 2h 33m 7s
A media de tempo de cada partida foi de 38.0m 16.75s


In [17]:
def get_match_detail(match_ids):
    url = f"https://api.opendota.com/api/matches/{match_id}"
    response = requests.get(url)
    return response.json()

In [18]:
url = f'https://api.opendota.com/api/teams/{tl_id}/players'
response = requests.get(url) 
players1 = response.json()
players_name = []
for player in players1:
    account_id = player.get('name')
    in_team = player.get("is_current_team_member")
    if account_id and in_team == True:
        players_name.append(account_id)

In [19]:
mean_kda = {player_name: [] for player_name in players_name} 
mean_kills = {player_name: [] for player_name in players_name} 
mean_deaths = {player_name: [] for player_name in players_name} 
mean_assists = {player_name: [] for player_name in players_name} 

for match_id in match_ids:
    match_data = get_match_detail(match_id)
    players = match_data['players']
    
    for player in players:
        jogador_nome = player.get('name', 'Desconhecido')
        if jogador_nome in players_name:
            kills = player['kills']
            deaths = player['deaths']
            assists = player['assists']
            kda = (kills + assists) / (deaths if deaths > 0 else 1) 

            mean_kda[jogador_nome].append(kda)
            mean_kills[jogador_nome].append(kills)
            mean_deaths[jogador_nome].append(deaths)
            mean_assists[jogador_nome].append(assists)


for jogador_nome in players_name:
    kda_lista = mean_kda[jogador_nome]
    kills_lista = mean_kills[jogador_nome]
    deaths_lista = mean_deaths[jogador_nome]
    assists_lista = mean_assists[jogador_nome]
    
    kda_medio = sum(kda_lista) / len(kda_lista) if kda_lista else 0
    kills_medio = sum(kills_lista) / len(kills_lista) if kills_lista else 0
    deaths_medio = sum(deaths_lista) / len(deaths_lista) if deaths_lista else 0
    assists_medio = sum(assists_lista) / len(assists_lista) if assists_lista else 0
    if kda_medio > 0:    
        print(f"O jogador {jogador_nome} tem uma média de {kda_medio:.2f} de KDA.")
        print(f" - Média de kills: {kills_medio:.2f}")
        print(f" - Média de mortes: {deaths_medio:.2f}")
        print(f" - Média de assistências: {assists_medio:.2f}")

O jogador Insania  tem uma média de 2.77 de KDA.
 - Média de kills: 2.25
 - Média de mortes: 6.25
 - Média de assistências: 12.75
O jogador m1CKe tem uma média de 8.03 de KDA.
 - Média de kills: 7.75
 - Média de mortes: 3.75
 - Média de assistências: 8.25
O jogador Boxi tem uma média de 3.49 de KDA.
 - Média de kills: 4.25
 - Média de mortes: 6.75
 - Média de assistências: 15.75
O jogador Nisha tem uma média de 5.04 de KDA.
 - Média de kills: 5.00
 - Média de mortes: 3.25
 - Média de assistências: 11.25
O jogador El SaberLightO tem uma média de 5.38 de KDA.
 - Média de kills: 4.75
 - Média de mortes: 4.25
 - Média de assistências: 15.50


In [20]:
def get_hero_mapping():
    url = "https://api.opendota.com/api/heroes"
    response = requests.get(url)
    heroes_data = response.json()
    
    hero_mapping = {hero['id']: hero['localized_name'] for hero in heroes_data}
    return hero_mapping

hero_mapping = get_hero_mapping()

In [21]:
hero_counter = collections.Counter()

for match_id in match_ids:
    match_data = get_match_detail(match_id)
    players = match_data['players']
    
    for player in players:
        hero_id = player['hero_id']
        hero_counter[hero_id] += 1

top_heroes = hero_counter.most_common(5)

print("Heróis que mais apareceram nas últimas partidas:")
for hero_id, count in top_heroes:
    hero_name = hero_mapping.get(hero_id, "Desconhecido")
    print(f"Herói {hero_name} apareceu {count} vezes.")

Heróis que mais apareceram nas últimas partidas:
Herói Dark Willow apareceu 3 vezes.
Herói Marci apareceu 3 vezes.
Herói Clockwerk apareceu 3 vezes.
Herói Muerta apareceu 3 vezes.
Herói Earth Spirit apareceu 2 vezes.
